### Note!
you have to install ipywidgets using `pip install ipywidgets`.

In [6]:
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from ipyvtklink.viewer import ViewInteractiveWidget

import ipywidgets as widgets
from IPython.display import display

In [7]:
hip_path = "/Users/nathanneeteson/Library/CloudStorage/OneDrive-UniversityofCalgary/MDSC68903_ImageData/Hip"
thrx_path = "/Users/nathanneeteson/Library/CloudStorage/OneDrive-UniversityofCalgary/MDSC68903_ImageData/Thorax"
head_path = "/Users/nathanneeteson/Library/CloudStorage/OneDrive-UniversityofCalgary/MDSC68903_ImageData/head.nii"

In [8]:
class vtk_pipeline:
    
    def __init__(self, path, filetype, window, level, window_sg, level_sg, zslice, render_size = 300):
        self.path = path
        self.filetype = filetype
        self.window = window
        self.level = level
        self.render_size = render_size
        self.slice = zslice
        
        self.window_sg = window_sg
        self.level_sg = level_sg

        self.out, self.outport, self.dim, self.img_np = self.image_reader()
                
        self.resizers = {}
        self.mappers = {}
        self.actors = {}
        
        self.segmentation_color = (1.0, 0.0, 0.0) #RGB value
        self.segmentation_opacity = 1.0
        self.segmented_image = None
        self.segmented_array = None
        
        self.renderer = None
        self.render_window = None
        self.interactor = None
        
    def image_reader(self):   
        if self.filetype.lower() == 'dicom':  # Load a DICOM image from file into a vtk pipeline.
            img_reader = vtk.vtkDICOMImageReader()
            img_reader.SetDirectoryName(self.path)
            img_reader.Update()
            out = img_reader.GetOutput()
            outport = img_reader.GetOutputPort()
            dim = out.GetDimensions()
            
        elif self.filetype.lower() == 'niftii': # Load a NIFTI image from file into a vtk pipeline.
            img_reader = vtk.vtkNIFTIImageReader()
            img_reader.SetFileName(self.path)
            img_reader.Update()
            out = img_reader.GetOutput()
            outport = img_reader.GetOutputPort()
            dim = out.GetDimensions()
            
        img_np = vtk_to_numpy(out.GetPointData().GetScalars()).reshape(dim,order='F')
        
        print("image read")
        
        return out, outport, dim, img_np
        
    def global_thresholder(self, lower, upper):
        
        segmentation_array = np.where((self.img_np >= lower) & (self.img_np <= upper), 1, 0)
        print("segmentation done")

        return segmentation_array     
    
    def dilation(self, segment_arr):
        kernel = np.ones((3,3,3))
        kernel_size = kernel.shape[0]

        padded = np.pad(segment_arr, [(1, 1), (1, 1), (1, 1)], mode='constant', constant_values = 0)
        
        output_array = np.zeros_like(segment_arr)
        
        print("start dilation")
        for (i, j, k), v in np.ndenumerate(segment_arr):
            region = padded[i:i+kernel_size, j:j+kernel_size, k:k+kernel_size]
            output_array[i, j, k] = np.max(kernel*region)

        return output_array
    
    def erosion(self,arr):
        kernel = np.ones((3,3,3))
        kernel_size = kernel.shape[0]
        
        padded = np.pad(arr, [(1, 1), (1, 1), (1, 1)], mode='constant', constant_values = 0)
        
        output_array = np.zeros_like(arr)
        print("start erosion")
        for (i, j, k), v in np.ndenumerate(arr):
                region = padded[i:i+kernel_size, j:j+kernel_size, k:k+kernel_size]
                output_array[i, j, k] = np.min(kernel*region)
        
        return output_array
                    
    def gaussian_filter(self):
        dimensionality = 3
        sd = 1.5
        radius_factor = 1
        
        gaussian = vtk.vtkImageGaussianSmooth()
        gaussian.SetDimensionality(dimensionality)
        gaussian.SetStandardDeviation(sd)
        gaussian.SetRadiusFactor(radius_factor)
        gaussian.SetInputData(self.out)
        gaussian.Update()
        
        return gaussian        
        
    def create_mousewheel_callbacks(self,mapper):
        
        def move_forward(obj=None,event=None):
            mapper.SetZSlice(mapper.GetZSlice()+1)

        def move_backward(obj=None,event=None):
            mapper.SetZSlice(mapper.GetZSlice()-1)

        return move_forward, move_backward
    
    def Keypress(self, obj, event):
        
        # Define keyboard commands
        key = obj.GetKeySym()
        if key == "Up":
            self.segmentation_opacity = self.segmentation_opacity + 0.05
            if self.segmentation_opacity > 1:
                self.segmentation_opacity = 1.0
        elif key == "Down":
            self.segmentation_opacity = self.segmentation_opacity - 0.05
            if self.segmentation_opacity < 0:
                self.segmentation_opacity = 0.0

        # Update window
        self.actors["segmented"].GetProperty().SetOpacity(self.segmentation_opacity)
        pass
    
    def create_resizer_mapper_actor(self, data_output, window, level):

        resizer = vtk.vtkImageResize()
        resizer.SetInputData(data_output)
        resizer.Update()
    
        original_dims = np.asarray(resizer.GetOutput().GetDimensions())
        new_dims = list( (original_dims*(self.render_size/original_dims[0:1].max())).astype(int) )

        resizer.SetOutputDimensions(*new_dims)

        mapper = vtk.vtkImageMapper()
        mapper.SetInputData(resizer.GetOutput())
        mapper.SetColorWindow(window)
        mapper.SetColorLevel(level)
        mapper.SetZSlice(self.slice)

        actor = vtk.vtkActor2D()
        actor.SetMapper(mapper)

        return resizer, mapper, actor
            
    def render(self, lower_bound, upper_bound, g_filter=False, dilation=True, erosion=False):
        
        self.resizers["image"], self.mappers["image"], self.actors['image'] = self.create_resizer_mapper_actor(self.out, self.window, self.level)
        
        if g_filter:
            self.out = self.gaussian_filter().GetOutput()
            self.img_np = vtk_to_numpy(self.out.GetPointData().GetScalars()).reshape(self.dim,order='F')
            self.resizers["image"], self.mappers["image"], self.actors['image'] = self.create_resizer_mapper_actor(self.out, self.window, self.level)
            
        print('image mapper created')
        
        self.segmented_array = self.global_thresholder(lower_bound, upper_bound)
        
        if dilation:
            self.segmented_array = self.dilation(self.segmented_array)
            
        if erosion:
            self.segmented_array = self.erosion(self.segmented_array)
        
        self.segmented_image = vtk.vtkImageData()
        self.segmented_image.DeepCopy(self.out) # here is how to copy a vtkImageData
        self.segmented_image.GetPointData().SetScalars(numpy_to_vtk(num_array = self.segmented_array.ravel(order = 'F'), deep = True))
        
    
        print("segmentation done.")
        
        out_value = 0
        in_value = 1
        lookup_table = vtk.vtkLookupTable()
        lookup_table.SetNumberOfTableValues(2)
        lookup_table.SetRange(0.0, 1.0)
        lookup_table.SetTableValue(out_value, 0.0, 0.0, 0.0, 0.0)  # label 0 is transparent
        lookup_table.SetTableValue(in_value, *self.segmentation_color, self.segmentation_opacity)  # label 1 is red and opaque
        lookup_table.Build()

        # Color your segmentation
        color_map = vtk.vtkImageMapToColors()
        color_map.SetInputData(self.segmented_image)
        color_map.SetLookupTable(lookup_table)
        color_map.PassAlphaToOutputOn()
        color_map.Update()
        
        
        self.resizers["segmented"], self.mappers["segmented"], self.actors['segmented'] = self.create_resizer_mapper_actor(color_map.GetOutput(), self.window_sg, self.level_sg)


        self.render_window = vtk.vtkRenderWindow()
        self.render_window.SetOffScreenRendering(True)
        self.render_window.SetSize(self.render_size, self.render_size)

        self.renderer = vtk.vtkRenderer()
        self.renderer.ResetCamera()       
        self.render_window.AddRenderer(self.renderer)

        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetRenderWindow(self.render_window)
        
        for actor in self.actors.values():
            self.renderer.AddActor(actor)
            
        
        # add callbacks to the interactor
        for mapper in self.mappers.values():
            fwd, bwd = self.create_mousewheel_callbacks(mapper)
            self.interactor.AddObserver('MouseWheelForwardEvent', fwd)
            self.interactor.AddObserver('MouseWheelBackwardEvent', bwd)
            self.interactor.AddObserver("KeyPressEvent", self.Keypress)
            
        self.render_window.Render()
        self.interactor.Initialize()

# Experiment
1. Gaussian filtering is applied on all three datasets
2. Global Thresholding is applied on each dataset with appropriate bounds
3. For head dataset, I applied closing operator. For thorax and head dataset, I just applied dilation(only because it takes so long to do both on these datasets!)

## Thorax
Applied Gaussian Filtering, global threshold, and dilation

In [9]:
thorax = vtk_pipeline(path=thrx_path, filetype='dicom', window=1000, level=40, window_sg=1, level_sg=1, zslice=150, render_size=500)
thorax.render(lower_bound=400, upper_bound=1000, g_filter=True, dilation=True, erosion=False)

ViewInteractiveWidget(thorax.render_window)


image read
image mapper created
segmentation done
start dilation
segmentation done.


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=500)

Thorax dataset - bone segmentation(lower_bound:400, upper_bound:1000) - window:1000, level:40, slice:150

# Hip
Applied Gaussian Filtering, global threshold, and dilation

In [10]:
hip = vtk_pipeline(path=hip_path, filetype='dicom', window=400, level=40, window_sg=1, level_sg=1, zslice=150, render_size=500)
hip.render(lower_bound=200, upper_bound=1500, g_filter=True, dilation=True, erosion=False)

ViewInteractiveWidget(hip.render_window)

image read
image mapper created
segmentation done
start dilation
segmentation done.


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=500)

Hip dataset - bone segmentation (lower_bound:200, upper_bound:1500) with Gaussian filtering, global threshold, and dilation - window:400, level:40, slice:150

# Head
Applied Gaussian filtering, global threshold, dilation and erosion(closing)

In [11]:
head = vtk_pipeline(path=head_path, filetype='niftii', window=700, level=200, window_sg=1, level_sg=1, render_size=300, zslice=150)

head.render(lower_bound=500, upper_bound=900, g_filter=True, dilation=True, erosion=True)

ViewInteractiveWidget(head.render_window)

image read
image mapper created
segmentation done
start dilation
start erosion
segmentation done.


ViewInteractiveWidget(height=300, layout=Layout(height='auto', width='100%'), width=300)

Head dataset - White matter segmentation (lower_bound:500, upper_bound:900) with Gaussian filtering, global threshold, dilation and erosion(closing) - window:1000, level:50, slice:150

# User Input

Here, you can choose what dataset you want to segment. You can specify window, level, slice, upper and lower bound for segmentation. You can choose if you want to apply Gaussian filtering or not. You can also choose whether you want to apply dilation, erosion, or both (for sake of simplicity, just closing operator).

In [12]:
image_options = [ ('Head', head_path), ('Hip', hip_path), ('Thorax', thrx_path)]
image = widgets.Dropdown(
    options=image_options,
    description='Image:',
)
filetype = widgets.Dropdown(
        options = ['dicom','niftii'],
        value = 'niftii',
        description = 'File Type: '
)
window = widgets.IntText(description="Window:", value=700)
level =  widgets.IntText(description="level:", value=200)
zslice = widgets.IntText(description="slice:", value=150)
lower = widgets.IntText(description="lower bound for segmentation:", value=500)
upper = widgets.IntText(description="upper bound for segmentation:", value=900)
tofilter = widgets.Checkbox(
    value=True,
    description='apply Gaussian filter',
)

dilation = widgets.Checkbox(
    value=True,
    description='apply dilation',
)
erosion = widgets.Checkbox(
    value=True,
    description='apply erosion',
)

display(image, filetype, window, level, zslice, lower, upper, tofilter, dilation, erosion)

Dropdown(description='Image:', options=(('Head', '/Users/nathanneeteson/Library/CloudStorage/OneDrive-Universi…

Dropdown(description='File Type: ', index=1, options=('dicom', 'niftii'), value='niftii')

IntText(value=700, description='Window:')

IntText(value=200, description='level:')

IntText(value=150, description='slice:')

IntText(value=500, description='lower bound for segmentation:')

IntText(value=900, description='upper bound for segmentation:')

Checkbox(value=True, description='apply Gaussian filter')

Checkbox(value=True, description='apply dilation')

Checkbox(value=True, description='apply erosion')

In [13]:
obj = vtk_pipeline(path=image.value, filetype=filetype.value, window=window.value, level=level.value, zslice=zslice.value, window_sg=1, level_sg=1, render_size=500)

obj.render(lower_bound=lower.value, upper_bound=upper.value, g_filter=tofilter.value, dilation= dilation.value, erosion= erosion.value)

ViewInteractiveWidget(obj.render_window)

image read
image mapper created
segmentation done
start dilation
start erosion
segmentation done.


ViewInteractiveWidget(layout=Layout(height='auto', width='100%'), width=500)